# Humanoid Walker with Arm following Leg Traj

### Import Libraries

In [1]:
import pydot
import numpy as np
from IPython.display import SVG, display

from pydrake.all import Simulator, DiagramBuilder, AddMultibodyPlantSceneGraph,\
                        Parser, RigidTransform, MeshcatVisualizer, MeshcatVisualizerParams, \
                        ConstantVectorSource, ConstantValueSource, PiecewisePolynomial,\
                        AbstractValue, HalfSpace, CoulombFriction, LeafSystem, BasicVector, \
                        MultibodyPlant, InverseDynamicsController
import footstep_planner
import osc
import importlib
import input_output_transformer
importlib.reload(osc)
importlib.reload(footstep_planner)
importlib.reload(input_output_transformer)
from osc import OperationalSpaceWalkingController, OscGains
from input_output_transformer import DecomposeStates, CombineControlInputs

/shared-libs/python3.8/py/lib/python3.8/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.17.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Start Meshcat

In [2]:
from pydrake.all import StartMeshcat

meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://4a6731e0-80e9-4a54-8f6c-6ed1aa743f6a.deepnoteproject.com/7000/


### Add Humanoid Walker URDF as the Simulation Plant

In [3]:
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
X_WG = HalfSpace.MakePose(np.array([0,0, 1]), np.zeros(3,))
plant.RegisterCollisionGeometry(
    plant.world_body(), 
    X_WG, HalfSpace(), 
    "collision", 
    CoulombFriction(1.0, 1.0)
)
# Note that we parse into both the plant and the scene_graph here.
Parser(plant, scene_graph).AddModels("urdf/humanoid_walker.urdf")
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base"))
plant.Finalize()

# Add MeshcatVisualizer for visualization
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

### Use Double Pendulum URDF to calculate PID Control Input for the Arms

In [4]:
# Secondary MultibodyPlant for control
control_plant = MultibodyPlant(time_step=1e-4)
control_model = Parser(control_plant).AddModels("urdf/double_pendulum.urdf")[0]
control_plant.Finalize()

# Define PID gains and add the inverse dynamics controller
kp = [100] * control_plant.num_positions()
ki = [1] * control_plant.num_positions()
kd = [20] * control_plant.num_positions()
left_arm_controller = builder.AddSystem(InverseDynamicsController(control_plant, kp, ki, kd, False))
left_arm_controller.set_name("left_arm_controller")
right_arm_controller = builder.AddSystem(InverseDynamicsController(control_plant, kp, ki, kd, False))
right_arm_controller.set_name("right_arm_controller")

### Calculate Arm Trajectory to follow

In [5]:
# Define a custom system for generating the desired trajectory
class ArmTrajectorySource(LeafSystem):
    def __init__(self):
        super().__init__()
        
        # Declare an input port for the full plant state (22 states)
        self.input_port_index = self.DeclareVectorInputPort("full_state", BasicVector(22)).get_index()

        # Declare an output port for the desired state (position and velocity)
        self.left_arm_traj_output_index = self.DeclareVectorOutputPort(
            "left_arm_traj", BasicVector(4), self.CalcLeftArm).get_index()
        self.right_arm_traj_output_index = self.DeclareVectorOutputPort(
            "right_arm_traj", BasicVector(4), self.CalcRightArm).get_index()

    def CalcLeftArm(self, context, output):
        theta1 = self.get_input_port(self.input_port_index).Eval(context)[9] # 10th state (right hip pos)
        theta1_dot = self.get_input_port(self.input_port_index).Eval(context)[20] # 21th state (right hip vel)
        theta2 = 0
        theta2_dot = 0

        offset = np.arccos(0.6)

        output.SetFromVector([theta1 + offset, theta2, theta1_dot, theta2_dot])

    def CalcRightArm(self, context, output):
        theta1 = self.get_input_port(self.input_port_index).Eval(context)[7] # 8th state (left hip pos)
        theta1_dot = self.get_input_port(self.input_port_index).Eval(context)[18] # 19th state (left hip vel)
        theta2 = 0
        theta2_dot = 0

        offset = np.arccos(0.6)

        output.SetFromVector([theta1 + offset, theta2, theta1_dot, theta2_dot])

### Decompose States

In [6]:
states_decomposer = DecomposeStates()
builder.AddSystem(states_decomposer)

# wire plant state to state decomposer
builder.Connect(plant.get_state_output_port(), 
                states_decomposer.get_input_port(states_decomposer.input_port_index))

### Connect Arm States to Arm PID Controller

In [7]:
builder.Connect(
    states_decomposer.get_output_port(
        states_decomposer.left_arm_states_output_port_index
        ),  
    left_arm_controller.get_input_port_estimated_state(),
)
builder.Connect(
    states_decomposer.get_output_port(
        states_decomposer.right_arm_states_output_port_index
        ),  
    right_arm_controller.get_input_port_estimated_state(),
)

### Connect Arm Desired Trajectory to Arm PID Controller

In [8]:
# Create and add the trajectory source to generate the desired trajectory
arm_trajectory_source = builder.AddSystem(ArmTrajectorySource())

builder.Connect(plant.get_state_output_port(), 
                arm_trajectory_source.get_input_port(arm_trajectory_source.input_port_index))

builder.Connect(
    arm_trajectory_source.get_output_port(
        arm_trajectory_source.left_arm_traj_output_index
        ), 
    left_arm_controller.get_input_port_desired_state()
)
builder.Connect(
    arm_trajectory_source.get_output_port(
        arm_trajectory_source.right_arm_traj_output_index
        ), 
    right_arm_controller.get_input_port_desired_state()
)

### Set up OSC and Footstep Planner

In [9]:
# Build the controller diagram
Kp = np.diag([100, 0, 100])
Kd = np.diag([10, 0, 10])
W = np.diag([1, 0, 1])

Wcom = np.zeros((3,3))
Wcom[2,2] = 1

# gains for humanoid with arms fixed
gains = OscGains(
        Kp, Kd, Wcom,
        Kp, Kd, W,
        60 * np.eye(1), 6 * np.eye(1), np.eye(1),
        0.00001
    )

walking_speed = 0.2 # walking speed in m/s for humanoid with arms fixed

osc = builder.AddSystem(OperationalSpaceWalkingController(gains))
planner = builder.AddSystem(footstep_planner.LipTrajPlanner())
speed_src = builder.AddSystem(ConstantVectorSource(np.array([walking_speed])))
base_traj_src = builder.AddSystem(
    ConstantValueSource(AbstractValue.Make(PiecewisePolynomial(np.zeros(1,))))
)

### Connect Lower Body States to Footstep Planner and OSC

In [10]:
builder.Connect(states_decomposer.get_output_port(
                    states_decomposer.lower_body_states_output_port_index
                    ),   
                planner.get_state_input_port())
builder.Connect(speed_src.get_output_port(), 
                planner.get_walking_speed_input_port())

builder.Connect(states_decomposer.get_output_port(
                    states_decomposer.lower_body_states_output_port_index
                    ), 
                osc.get_state_input_port()) 
builder.Connect(planner.get_swing_foot_traj_output_port(), 
                osc.get_traj_input_port("swing_foot_traj"))
builder.Connect(planner.get_com_traj_output_port(), 
                osc.get_traj_input_port("com_traj"))
builder.Connect(base_traj_src.get_output_port(), 
                osc.get_traj_input_port("base_joint_traj"))

### Combine all Control Inputs and wire with Plant Actuation Input

In [11]:
control_input_combiner = CombineControlInputs()
builder.AddSystem(control_input_combiner)

builder.Connect(osc.get_output_port(), 
                control_input_combiner.get_input_port(control_input_combiner.osc_input_port_index))
builder.Connect(left_arm_controller.get_output_port(), 
                control_input_combiner.get_input_port(control_input_combiner.left_arm_control_input_port_index))
builder.Connect(right_arm_controller.get_output_port(), 
                control_input_combiner.get_input_port(control_input_combiner.right_arm_control_input_port_index))

builder.Connect(control_input_combiner.get_output_port(control_input_combiner.combined_control_input_output_port_index), 
                plant.get_actuation_input_port())

### Build Diagram

In [12]:
# Build the diagram
diagram = builder.Build()
diagram.set_name("humanoid_walking_controller")

### Run Simulation

In [ ]:
sim_time = 1.5
simulator = Simulator(diagram)
simulator.Initialize()
simulator.set_target_realtime_rate(1)

# Set the robot state
plant_context = diagram.GetMutableSubsystemContext(
    plant, simulator.get_mutable_context())

q = np.zeros((plant.num_positions(),))
q[1] = 0.8
theta = -np.arccos(q[1])
q[7] = theta
q[8] = -2 * theta
q[9] = theta
q[10] = -2 * theta
plant.SetPositions(plant_context, q)

# Start recording the animation
meshcat.StartRecording()
simulator.AdvanceTo(sim_time)
meshcat.StopRecording()
meshcat.PublishRecording()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4a6731e0-80e9-4a54-8f6c-6ed1aa743f6a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>